In [198]:
# conda install -c anaconda pandas
# conda install -c anaconda nltk
# conda install -c conda-forge matplotlib
# conda install -c conda-forge spacy
# conda install -c anaconda scikit-learn spacy download en_core_web_sm
# conda install -c conda-forge spacy download en_core_web_sm
# conda install -c conda-forge spacy-model-en_core_web_sm
# conda install pip
#!pip install pywsd
# import nltk
# nltk.download('averaged_perceptron_tagger')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('words')
#!conda install -c anaconda seaborn --yes


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/sdatta/anaconda3/envs/py36

  added / updated specs:
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    patsy-0.5.1                |           py36_0         380 KB  anaconda
    seaborn-0.9.0              |           py36_0         379 KB  anaconda
    statsmodels-0.9.0          |   py36h035aef0_0         9.0 MB  anaconda
    ------------------------------------------------------------
                                           Total:         9.8 MB

The following NEW packages will be INSTALLED:

  patsy              anaconda/linux-64::patsy-0.5.1-py36_0
  seaborn            anaconda/linux-64::seaborn-0.9.0-py36_0
  st

In [508]:
import json
import itertools
import pandas as pd
import ast
import string
import operator
from collections import defaultdict
import re
import os
import nltk
import math
import json
from multiprocessing import Pool
from os import listdir
from os.path import isfile, join
from os import walk
import matplotlib.pyplot as plt
%matplotlib inline
# from lxml import html
from nltk.tokenize import PunktSentenceTokenizer
import nltk
#from  nltk.parse.corenlpnltk.pa  import CoreNLPParser
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
import spacy    
from spacy import displacy
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns; sns.set()
import numpy as np
import pickle

In [23]:
#Spacy
#conda install -c conda-forge spacy-model-en_core_web_sm
import en_core_web_sm

In [24]:
nlp = en_core_web_sm.load()


In [31]:
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words as nltk_words

lemmatizer = WordNetLemmatizer() 
#nltk.download('wordnet')
ps = PorterStemmer()
stopWords = set(stopwords.words('english'))
stopWords.update(['[',']','the','no'])

In [8]:
from pywsd.lesk import simple_lesk
#from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim
from pywsd.lesk import cached_signatures
from nltk.corpus import wordnet as wn

In [9]:
LIMIT_TOKEN_SIZE=10000

In [10]:
in_dir="./input"
out_dir="./output"


In [13]:
path_to_json = '/home/sdatta/data/wikipedia_squad'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]


In [357]:
flag_break=0
path_to_json = '/home/sdatta/data/wikipedia_squad'

df_data=pd.DataFrame(index=None, columns=['data', 'version'])

for path, subdirs, files in os.walk(path_to_json):
    if flag_break==1:
        break
    for file_num, name in enumerate(files):
        if flag_break==1:
            break
        if not name.startswith('wiki'):
#         if not ( name.startswith('wiki_00') #or  name.startswith('wiki_01') 
#                ):
            continue
        dict_temp=dict()
        file_path = os.path.join(path, name)
        file_name_out = "/".join(file_path.split('/')[-2:])
        dict_temp = pd.read_json(file_path)
        for data_num, ele1 in enumerate(dict_temp['data']):
            if flag_break==1:
                break
            dict_temp['data'][data_num]['filename']=file_name_out
        df_data=df_data.append(pd.DataFrame(dict_temp), ignore_index=True)
        print(file_name_out)


AB/wiki_16
AB/wiki_39
AB/wiki_47
AB/wiki_61
AB/wiki_90
AB/wiki_15
AB/wiki_24
AB/wiki_50
AB/wiki_57
AB/wiki_28
AB/wiki_79
AB/wiki_96
AB/wiki_75
AB/wiki_32
AB/wiki_87
AB/wiki_30
AB/wiki_95
AB/wiki_31
AB/wiki_01
AB/wiki_63
AB/wiki_56
AB/wiki_76
AB/wiki_99
AB/wiki_67
AB/wiki_33
AB/wiki_06
AB/wiki_72
AB/wiki_80
AB/wiki_59
AB/wiki_14
AB/wiki_42
AB/wiki_02
AB/wiki_19
AB/wiki_65
AB/wiki_81
AB/wiki_55
AB/wiki_74
AB/wiki_13
AB/wiki_34
AB/wiki_84
AB/wiki_23
AB/wiki_36
AB/wiki_46
AB/wiki_68
AB/wiki_51
AB/wiki_04
AB/wiki_00
AB/wiki_66
AB/wiki_77
AB/wiki_18
AB/wiki_48
AB/wiki_91
AB/wiki_10
AB/wiki_26
AB/wiki_60
AB/wiki_45
AB/wiki_85
AB/wiki_73
AB/wiki_70
AB/wiki_62
AB/wiki_97
AB/wiki_25
AB/wiki_49
AB/wiki_71
AB/wiki_58
AB/wiki_52
AB/wiki_86
AB/wiki_09
AB/wiki_43
AB/wiki_44
AB/wiki_21
AB/wiki_92
AB/wiki_27
AB/wiki_35
AB/wiki_38
AB/wiki_41
AB/wiki_29
AB/wiki_69
AB/wiki_64
AB/wiki_11
AB/wiki_88
AB/wiki_82
AB/wiki_89
AB/wiki_40
AB/wiki_83
AB/wiki_07
AB/wiki_93
AB/wiki_03
AB/wiki_08
AB/wiki_37
AB/wiki_98

In [17]:
#Testing
df_data['data'][1]['paragraphs'][68]['context']

'The Internet has helped autistic individuals bypass nonverbal cues and emotional sharing that they find difficult to deal with, and has given them a way to form online communities and work remotely. Societal and cultural aspects of autism have developed: some in the community seek a cure, while others believe that autism is simply another way of being.'

In [358]:
#Concatening all context to all_context
for data_num, ele1 in enumerate(df_data['data']):
    title=ele1['title']
    df_data['data'][data_num]['all_context']=''
    for para_num, ele2 in enumerate(ele1['paragraphs']):
        df_data['data'][data_num]['all_context']=df_data['data'][data_num]['all_context']+ele2['context']


In [359]:
#New dataframe with title, row_num , all_context
df_data_2=pd.DataFrame()
for data_num, ele1 in enumerate(df_data['data']):
    title=df_data['data'][data_num]['title']
    row_num=data_num
    all_context=df_data['data'][data_num]['all_context']
    dict_tmp={'title':title,'row_num':row_num,'raw_context':all_context}    
    # The dictionary needs to be in [] brackets
    df_data_2=df_data_2.append(pd.DataFrame([dict_tmp]), ignore_index=True)


In [360]:
#validation
df_data_2.shape

(10302, 3)

In [37]:
#For development
df_data_3=pd.DataFrame()
df_data_3=df_data_2.iloc[0:5]


In [23]:
df_data_3.shape

(5, 3)

In [91]:
def f_tokenize(in_raw_context: str):
    tokenize_cleaned=[lemmatizer.lemmatize(ele) for ele in word_tokenize(in_raw_context.lower()) if (ele not in stopWords and ele.isalpha())]
    #tokenize_cleaned=[ele for ele in tokenize_cleaned if ele in nltk_words.words()]
    return " ".join(tokenize_cleaned)

In [97]:
#NOT USED
# def get_cosine_sim(*strs): 
#     vectors = [t for t in get_vectors(*strs)]
#     return cosine_similarity(vectors)
    


In [98]:
#NOT USED
# def get_vectors(*strs):
#     text = [t for t in strs]
#     vectorizer = CountVectorizer(text)
#     vectorizer.fit(text)
#     return vectorizer.transform(text).toarray()


In [92]:
df_data_3['tokenized_context']=df_data_3['raw_context'].apply(f_tokenize)

/home/sdatta/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
df_data_3['tokenized_context']

0    anarchism political philosophy advocate societ...
1    autism developmental disorder characterized di...
2    albedo meaning measure diffuse reflection sola...
3    named plural aes first letter first vowel mode...
4    alabama state southeastern region united state...
Name: tokenized_context, dtype: object

#TFIDF

In [99]:
tfidf = TfidfVectorizer() 
text_transformed = tfidf.fit_transform(df_data_3['tokenized_context'])


In [101]:
pairwise_similarity = text_transformed * text_transformed.T


In [105]:
df1=pd.DataFrame(pairwise_similarity.toarray())

In [107]:
df1

,0,1,2,3,4
0,1.000000,0.055508,0.019021,0.057081,0.119624
1,0.055508,1.000000,0.028461,0.050659,0.048557
2,0.019021,0.028461,1.000000,0.033505,0.032679
3,0.057081,0.050659,0.033505,1.000000,0.034654
4,0.119624,0.048557,0.032679,0.034654,1.000000


In [120]:
var1=df_data_3.iloc[0]

In [121]:
type(var1)

pandas.core.series.Series

In [123]:
df_data_3.columns

Index(['raw_context', 'row_num', 'title', 'tokenized_context'], dtype='object')

In [125]:
df_tmp=df_data_3.loc[0].to_frame()

In [117]:
type(df_tmp)

pandas.core.frame.DataFrame

In [118]:
df_tmp.columns

Int64Index([0], dtype='int64')

## Dataframe loc to Series to Dataframe

In [163]:
var1=df_data_3.loc[0]

In [164]:
var1

,raw_context,row_num,title,tokenized_context
0,Anarchism is a political philosophy that advoc...,0,Anarchism,anarchism political philosophy advocate societ...
0,Anarchism is a political philosophy that advoc...,0,Anarchism,anarchism political philosophy advocate societ...


In [136]:
var2=var1.to_frame().T

In [141]:
var2=var2.reset_index()

In [142]:
var2.columns

Index(['index', 'raw_context', 'row_num', 'title', 'tokenized_context'], dtype='object')

In [145]:
var2.drop(columns=['index'], inplace=True)

In [146]:
var2.columns

Index(['raw_context', 'row_num', 'title', 'tokenized_context'], dtype='object')

In [147]:
var2

,raw_context,row_num,title,tokenized_context
0,Anarchism is a political philosophy that advoc...,0,Anarchism,anarchism political philosophy advocate societ...


In [148]:
df_data_3=df_data_3.append(var2)

In [149]:
df_data_3

,raw_context,row_num,title,tokenized_context
0,Anarchism is a political philosophy that advoc...,0,Anarchism,anarchism political philosophy advocate societ...
1,Autism is a developmental disorder characteriz...,1,Autism,autism developmental disorder characterized di...
2,"Albedo () (, meaning 'whiteness') is the measu...",2,Albedo,albedo meaning measure diffuse reflection sola...
3,"A (named , plural ""As"", ""A's"", ""a""s, ""a's"" or ...",3,A,named plural aes first letter first vowel mode...
4,Alabama is a state in the southeastern region ...,4,Alabama,alabama state southeastern region united state...
0,Anarchism is a political philosophy that advoc...,0,Anarchism,anarchism political philosophy advocate societ...


In [150]:
text_transformed = tfidf.fit_transform(df_data_3['tokenized_context'])
pairwise_similarity = text_transformed * text_transformed.T


In [151]:
df1=pd.DataFrame(pairwise_similarity.toarray())

In [152]:
df1

,0,1,2,3,4,5
0,1.000000,0.054158,0.018739,0.057473,0.117219,1.000000
1,0.054158,1.000000,0.026240,0.045646,0.043454,0.054158
2,0.018739,0.026240,1.000000,0.032594,0.031129,0.018739
3,0.057473,0.045646,0.032594,1.000000,0.031933,0.057473
4,0.117219,0.043454,0.031129,0.031933,1.000000,0.117219
5,1.000000,0.054158,0.018739,0.057473,0.117219,1.000000


In [160]:
df1[df1>0.05]

,0,1,2,3,4,5
0,1.000000,0.054158,NaN,0.057473,0.117219,1.000000
1,0.054158,1.000000,NaN,NaN,NaN,0.054158
2,NaN,NaN,1.0,NaN,NaN,NaN
3,0.057473,NaN,NaN,1.000000,NaN,0.057473
4,0.117219,NaN,NaN,NaN,1.000000,0.117219
5,1.000000,0.054158,NaN,0.057473,0.117219,1.000000


In [526]:
set1=set()
set1.update([1,2,3,4,5])

In [529]:
set1.update([1,2,3,4,5])

In [530]:
set1

{1, 2, 3, 4, 5}

# Actual processing

In [422]:
def f_get_titles(in_row_idx: int, in_title_idx: list):
    print("Root title:",df_data_4.loc[in_row_idx]['title'])
    print("Similar titles are:")
    for ele in in_title_idx:
        print(df_data_4.loc[ele]['title'])

In [519]:
#MATCH_THRESHOLD=0.05
MATCH_THRESHOLD=0.1
def f_get_threshold_val_index(in_values):
    tmp_arr=np.array(in_values)[:-1]
    ret_value=np.nonzero(tmp_arr>=MATCH_THRESHOLD)[0].tolist()
    
    return ret_value


In [539]:
set_seencols=set()

def f_get_sim_titles(in_column_idx: list):
    
    global set_seencols

    list_titles=[]
    for ele in in_column_idx:
        if ele in set_seencols:
            continue
        else:
            list_titles.append(df_data_4.loc[ele]['title'])

    #Updating seen columns
    set_seencols.update(in_column_idx)
    
    ret_value=list_titles
    
    return ret_value

In [574]:
set_seencols_topnwords=set()
def get_top_n_words(list_corpus_idx: list, n=3, ):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    corpus=[]
    list_of_frequent_words=[]
    global set_seencols_topnwords
    
    list_corpus_idx_filtered = set(list_corpus_idx) - set_seencols_topnwords
    # for ele in list_corpus_idx:
    #     if ele in set_seencols_topnwords:
    #         continue
    #     else:
    #         corpus.append(df_data_4.iloc[ele]['tokenized_context'] )
    corpus.append(df_data_4.iloc[list(list_corpus_idx_filtered)]['tokenized_context'] )
    
    #Reason for corpus[0]
    #corpus=[]
    # corpus.append(df_data_4.iloc[[0,2,3]]['title'].values)
    # corpus
    # [array(['Hair', 'Hawker Harrier', 'Hergé'], dtype=object)]
    corpus=corpus[0]
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    list_of_frequent_words=[x[0] for x in words_freq]

    #Updating seen columns
    set_seencols.update(list_corpus_idx_filtered)

    #return words_freq[:n]
    return list_of_frequent_words[:n]


In [582]:
len(set_seencols)

10301

In [362]:
#For development
df_data_4=pd.DataFrame()
df_data_4=df_data_2.copy(deep=True)
df_data_4.shape

(10302, 3)

In [363]:
#Tokenizing and cleaning context
df_data_4['tokenized_context']=df_data_4['raw_context'].apply(f_tokenize)

In [364]:
# tfidf = TfidfVectorizer(analyzer='word', smooth_idf=False, sublinear_tf=False, norm=None,
#                         stop_words='english', ngram_range=(1,1), strip_accents='ascii',use_idf=True) 
tfidf = TfidfVectorizer() 
text_transformed = tfidf.fit_transform(df_data_4['tokenized_context'])
pairwise_similarity = text_transformed * text_transformed.T


In [520]:
#Finding similar columns
df_data_simi=pd.DataFrame(pairwise_similarity.toarray())
df_data_simi['sim_columns']=df_data_simi.apply(f_get_threshold_val_index,axis=1)

In [540]:
df_data_simi['sim_titles']=df_data_simi['sim_columns'].apply(f_get_sim_titles)

In [575]:
df_data_simi['common_words']=df_data_simi['sim_columns'].apply(get_top_n_words)

In [542]:
df_data_simi['sim_titles'].to_csv('/home/sdatta/work/sim_titles.txt', index=False)

/home/sdatta/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [576]:
df_data_simi['common_words'].to_csv('/home/sdatta/work/common_words.txt', index=False)

/home/sdatta/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [577]:
df_data_simi[['sim_titles','common_words']].to_csv('/home/sdatta/work/sim_titles_words.txt', sep='|', index=False)

In [578]:
df_data_simi['sim_columns'].to_csv('/home/sdatta/work/sim_columns.txt', index=False)

/home/sdatta/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [579]:
pickle.dump(df_data_4, open('/home/sdatta/work/df_data_4_1.0_20190416_1418.pkl','wb'))

In [580]:
pickle.dump(df_data_simi, open('/home/sdatta/work/df_data_simi_1.0_20190416_1418.pkl','wb'))

In [513]:
pickle.dump(df_data, open('/home/sdatta/work/df_data_20190416_0852.pkl','wb'))

In [511]:
pickle.dump(df_data_2, open('/home/sdatta/work/df_data_2_20190416_0852.pkl','wb'))

In [517]:
df_data_4.shape

(10302, 4)